<a href="https://colab.research.google.com/github/nounou-mimi/CROP-DISEASE-CODE/blob/main/classification/Classification_PlantifyDr_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploaded Kaggle.json

In [1]:
! pip install -q kaggle
! mkdir ~/.kaggle
%cd /content
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download lavaman151/plantifydr-dataset

/content
100% 2.57G/2.58G [00:36<00:00, 60.8MB/s]
100% 2.58G/2.58G [00:36<00:00, 76.2MB/s]


In [2]:
!unzip plantifydr-dataset.zip

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(3679).JPG  
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(368).JPG  
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(3680).JPG  
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(3681).JPG  
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(3682).JPG  
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(3683).JPG  
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(3684).JPG  
  inflating: PlantDiseasesDataset/Tomato/train/Tomato Yellow Leaf Curl Virus/TomatoYellowCurlVirus(3685).JPG  
  inflating: PlantDiseasesDataset/T

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,GlobalAveragePooling2D, Conv2D,InputLayer
from tensorflow.keras.layers import BatchNormalization,Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import InceptionResNetV2
import keras.backend as K
from tensorflow.keras.applications import MobileNetV2

from tensorflow import keras

from tensorflow.keras.models import load_model

In [4]:
TRAINING_DATA_PATH = '/content/PlantDiseasesDataset/Tomato/train'

In [8]:
IMAGE_SIZE = [224,224]
BATCH_SIZE = 64
LEARNING_RATE = 0.009
MOMENTUM = 0.9 
EPOCHS = 20
PATIENCE = 30
#CLASSES = len(glob('data/train'))

The following model is basically a MobileNet model that has been pretrained on PlantVillage Data (https://www.kaggle.com/thunder2901/leaf-disease-classification-mobilenet/data )

In [7]:
base_model=load_model('leaf-cnn.h5')

In [9]:
base_model.summary()

Model: "LeafDisease_MobileNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenet_1.00_224 (Function (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 38)                38950     
Total params: 3,267,814
Trainable params: 38,950
Non-trainable params: 3,228,864
_________________________________________________________________


In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train = train_datagen.flow_from_directory(
    TRAINING_DATA_PATH,
    target_size=(224,224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training') # set as training data

valid = train_datagen.flow_from_directory(
    TRAINING_DATA_PATH, # same directory as training data
    target_size=(224,224),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation') # set as validation data



Found 33415 images belonging to 10 classes.
Found 8348 images belonging to 10 classes.


In [11]:
inputShape = (224, 224, 3)
model=Sequential()

model.add(InputLayer(input_shape=inputShape))
base_model.layers[1].trainable=False
model.add(base_model.layers[1])
model.add(base_model.layers[2])
  
model.add(Dense(1024, activation='relu',name='fc1'))
model.add(Dropout(0.5,name='Dropout_1'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu',name='fc2'))
model.add(Dropout(0.3,name = 'Dropout'))
model.add(Dense(10,activation = 'softmax',name='output'))

In [12]:
adam=optimizers.Adam(learning_rate=0.001)
model.compile(
        loss = 'categorical_crossentropy',
        optimizer = adam,
        metrics = ['accuracy']
  )

In [ ]:
model.fit_generator(
    train,
    steps_per_epoch = train.samples // BATCH_SIZE,
    validation_data = valid, 
    validation_steps = valid.samples // BATCH_SIZE,
    epochs = 10)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
522/522 [==============================] - 556s 1s/step - loss: 0.5846 - accuracy: 0.8019 - val_loss: 0.3346 - val_accuracy: 0.8879
Epoch 2/10
522/522 [==============================] - 526s 1s/step - loss: 0.4167 - accuracy: 0.8566 - val_loss: 0.2887 - val_accuracy: 0.9048
Epoch 3/10
522/522 [==============================] - 524s 1s/step - loss: 0.3912 - accuracy: 0.8673 - val_loss: 0.2737 - val_accuracy: 0.9090
Epoch 4/10
522/522 [==============================] - 527s 1s/step - loss: 0.3600 - accuracy: 0.8778 - val_loss: 0.2508 - val_accuracy: 0.9186
Epoch 5/10
522/522 [==============================] - 520s 997ms/step - loss: 0.3413 - accuracy: 0.8834 - val_loss: 0.1941 - val_accuracy: 0.9362
Epoch 6/10
522/522 [==============================] - 539s 1s/step - loss: 0.3275 - accuracy: 0.8883 - val_loss: 0.2066 - val_accuracy: 0.9311
Epoch 7/10
522/522 [==============================] - 538s 1s/step - loss: 0.3155 - accuracy: 0.8930 - val_loss: 0.1803 - val_accuracy: 0.9